In [ ]:
import http.client
import json
import csv

In [ ]:
class Graph:

    # Do not modify
    def __init__(self, with_nodes_file=None, with_edges_file=None):
        """
        option 1:  init as an empty graph and add nodes
        option 2: init by specifying a path to nodes & edges files
        """
        self.nodes = []
        self.edges = []
        if with_nodes_file and with_edges_file:
            nodes_CSV = csv.reader(open(with_nodes_file))
            nodes_CSV = list(nodes_CSV)[1:]
            self.nodes = [(n[0],n[1]) for n in nodes_CSV]

            edges_CSV = csv.reader(open(with_edges_file))
            edges_CSV = list(edges_CSV)[1:]
            self.edges = [(e[0],e[1]) for e in edges_CSV]


    def add_node(self, id: str, name: str)->None:
        """
        add a tuple (id, name) representing a node to self.nodes if it does not already exist
        The graph should not contain any duplicate nodes
        """
        name = name.encode("ascii", "ignore").decode("ascii")
        t = (id, name)
        if t not in set(self.nodes):
            self.nodes.append(t)


    def add_edge(self, source: str, target: str)->None:
        """
        Add an edge between two nodes if it does not already exist.
        An edge is represented by a tuple containing two strings: e.g.: ('source', 'target').
        Where 'source' is the id of the source node and 'target' is the id of the target node
        e.g., for two nodes with ids 'a' and 'b' respectively, add the tuple ('a', 'b') to self.edges
        """
        t = (source, target)
        edges = set(self.edges)
        nodes = [t, tuple(reversed(t))]
        check = any(item in edges for item in nodes)
        if not check:
            self.edges.append(t)


    def total_nodes(self)->int:
        """
        Returns an integer value for the total number of nodes in the graph
        """
        return len(self.nodes)


    def total_edges(self)->int:
        """
        Returns an integer value for the total number of edges in the graph
        """
        return len(self.edges)


    def max_degree_nodes(self)->dict:
        """
        Return the node(s) with the highest degree
        Return multiple nodes in the event of a tie
        Format is a dict where the key is the node_id and the value is an integer for the node degree
        e.g. {'a': 8}
        or {'a': 22, 'b': 22}
        """
        d = {}
        for (a, b) in self.edges:
            if a not in d.keys():
                d[a] = 1
            else:
                d[a] += 1
            if b not in d.keys():
                d[b] = 1
            else:
                d[b] += 1
        output = {k : v for k, v in d.items() if v == max(d.values())}
        return output


    def print_nodes(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.nodes)


    def print_edges(self):
        """
        No further implementation required
        May be used for de-bugging if necessary
        """
        print(self.edges)


    # Do not modify
    def write_edges_file(self, path="edges.csv")->None:
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w')

        edges_file.write("source" + "," + "target" + "\n")

        for e in self.edges:
            edges_file.write(e[0] + "," + e[1] + "\n")

        edges_file.close()
        print("finished writing edges to csv")


    # Do not modify
    def write_nodes_file(self, path="nodes.csv")->None:
        """
        write all nodes out as .csv
        :param path: string
        :return: None
        """
        nodes_path = path
        nodes_file = open(nodes_path, 'w')

        nodes_file.write("id,name" + "\n")
        for n in self.nodes:
            nodes_file.write(n[0] + "," + n[1] + "\n")
        nodes_file.close()
        print("finished writing nodes to csv")



class  TMDBAPIUtils:

    # Do not modify
    def __init__(self, api_key:str):
        self.api_key=api_key


    def get_movie_cast(self, movie_id:str, limit:int=None, exclude_ids:list=None) -> list:
        """
        Get the movie cast for a given movie id, with optional parameters to exclude an cast member
        from being returned and/or to limit the number of returned cast members
        documentation url: https://developers.themoviedb.org/3/movies/get-movie-credits

        :param integer movie_id: a movie_id
        :param integer limit: number of returned cast members by their 'order' attribute
            e.g., limit=5 will attempt to return the 5 cast members having 'order' attribute values between 0-4
            If there are fewer cast members than the specified limit or the limit not specified, return all cast members
        :param list exclude_ids: a list of ints containing ids (not cast_ids) of cast members  that should be excluded from the returned result
            e.g., if exclude_ids are [353, 455] then exclude these from any result.
        :rtype: list
            return a list of dicts, one dict per cast member with the following structure:
                [{'id': '97909' # the id of the cast member
                'character': 'John Doe' # the name of the character played
                'credit_id': '52fe4249c3a36847f8012927' # id of the credit, ...}, ...]
                Note that this is an example of the structure of the list and some of the fields returned by the API. The result of the API call will include many more fields for each cast member.
        Important: the exclude_ids processing should occur prior to limiting output.
        """
        connection = http.client.HTTPSConnection("api.themoviedb.org")
        connection.request("GET", "/3/movie/{}/credits?api_key={}".format(movie_id, self.api_key))
        response = connection.getresponse()
        body = response.read().decode()
        movie_cast = json.loads(body)
        if movie_cast is not None:
            output = [d for d in movie_cast['cast']]
            if exclude_ids is not None:
                output = [d for d in output if d['id'] not in exclude_ids]
            if limit is not None:
                output = [d for d in output if d['order'] < limit]
        else:
            output = []
        return output


    def get_movie_credits_for_person(self, person_id:str, vote_avg_threshold:float=None)->list:
        """
        Using the TMDb API, get the movie credits for a person serving in a cast role
        documentation url: https://developers.themoviedb.org/3/people/get-person-movie-credits

        :param string person_id: the id of a person
        :param vote_avg_threshold: optional parameter to return the movie credit if it is >=
            the specified threshold.
            e.g., if the vote_avg_threshold is 5.0, then only return credits with a vote_avg >= 5.0
        :rtype: list
            return a list of dicts, one dict per movie credit with the following structure:
                [{'id': '97909' # the id of the movie credit
                'title': 'Long, Stock and Two Smoking Barrels' # the title (not original title) of the credit
                'vote_avg': 5.0 # the float value of the vote average value for the credit}, ... ]
        """
        connection = http.client.HTTPSConnection("api.themoviedb.org")
        connection.request("GET", "/3/person/{}/movie_credits?api_key={}&language=en-US".format(person_id, self.api_key))
        response = connection.getresponse()
        body = response.read().decode()
        person_credits = json.loads(body)
        if person_credits is not None:
            output = [
                {
                    k : v
                    for k, v in d.items()
                    if k in {'credit_id', 'title', 'vote_average', 'id'}
                }
                for d in person_credits['cast']
            ]
            if vote_avg_threshold is not None:
                output = [d for d in output if d['vote_average'] >= vote_avg_threshold]
        else:
            output = []
        return output

In [ ]:
api_key = 'e6c6d80656546a1dd359ab0b71ca6432'
top_node = {'id' : '5064', 'name' : 'Meryl Streep'}

In [ ]:
def combinations(lst, n):
    if n == 0:
        return [[]]
    l = []
    for i in range(0, len(lst)):
        m = lst[i]
        remainding_lst = lst[i + 1:]
        for p in combinations(remainding_lst, n - 1):
            l.append([m] + p)
    return l

def coactor_network(api_key:str, top_node:dict, iteration:int = 3) -> None:
    graph = Graph()
    graph.add_node(top_node['id'], top_node['name'])
    tmdb_api_utils = TMDBAPIUtils(api_key = api_key)
    iterable = graph.nodes
    for (person_id, name) in iterable:
        person_credits = tmdb_api_utils.get_movie_credits_for_person(person_id, vote_avg_threshold = 8.0)
        movie_cast = {}
        for d in person_credits:
            movie_cast[d['id']] = tmdb_api_utils.get_movie_cast(d['id'], limit = 3, exclude_ids = None)
        for movie_id, cast in movie_cast.items():
            for actor in cast:
                person_id1 = str(actor['id'])
                name1 = actor['name']
                if (person_id1, name1) not in set(graph.nodes):
                    graph.add_node(id=person_id1, name=name1)
                    iterable.
                combo_list = [str(d2['id']) for d2 in cast]
                if person_id not in set(combo_list):
                    combo_list.append(person_id)
                for t in combinations(combo_list, 2):
                    if t[0] != t[1] and (t[0], t[1]) not in set(graph.edges) and (t[1], t[0]) not in set(graph.edges):
                        graph.add_edge(source=t[0], target=t[1])
        if iteration == 0:
            break
        iteration -= 1
        
    graph.write_edges_file()
    graph.write_nodes_file()

coactor_network(api_key, top_node)

In [ ]:
test = []
t = 1 
s = 2
test.append((t, s))
test.pop
test

In [ ]:
graph = Graph()
graph.add_node(id='5064', name='Meryl Streep')
tmdb_api_utils = TMDBAPIUtils(api_key = api_key)

In [ ]:
print(type(graph.nodes))

In [ ]:
top_node_person_id = graph.nodes[0][0]

In [ ]:
person_credits = tmdb_api_utils.get_movie_credits_for_person(top_node_person_id, vote_avg_threshold = 8.0)

In [ ]:
movie_cast = {}
for d in person_credits:
    movie_cast[d['id']] = tmdb_api_utils.get_movie_cast(d['id'], limit = 3, exclude_ids = None)

In [ ]:
set(graph.nodes)

In [ ]:
set(graph.edges)

In [ ]:
def combinations(lst, n):  
    if n == 0: 
        return [[]]  
    l = [] 
    for i in range(0, len(lst)):     
        m = lst[i] 
        remainding_lst = lst[i + 1:] 
        for p in combinations(remainding_lst, n-1): 
            l.append([m] + p)   
    return l 

In [ ]:
for k, v in movie_cast.items():
    for d in v:
        person_id = str(d['id'])
        name = d['name']
        if (person_id, name) not in set(graph.nodes):
            graph.add_node(id = person_id, name = name)
        combo_list = [str(d2['id']) for d2 in v]
        if top_node_person_id not in set(combo_list):
            combo_list.append(top_node_person_id)
        for t in combinations(combo_list, 2):
                if t[0] != t[1] and (t[0], t[1]) not in set(graph.edges):
                    graph.add_edge(source = t[0], target = t[1])

In [ ]:
print(graph.edges)

In [ ]:
print(graph.nodes)

In [ ]:
assert len(set(graph.edges)) == len(graph.edges)
assert len(set(graph.nodes)) == len(graph.nodes)

In [ ]:
for (person_id, name) in graph.nodes[1:]:
    person_credits = tmdb_api_utils.get_movie_credits_for_person(person_id, vote_avg_threshold = 8.0)
    movie_cast = {}
    for d in person_credits:
        movie_cast[d['id']] = tmdb_api_utils.get_movie_cast(d['id'], limit = 3, exclude_ids = None)
    for k, v in movie_cast.items():
        for d1 in v:
            person_id1 = str(d1['id'])
            name1 = d1['name']
            if (person_id1, name1) not in set(graph.nodes):
                graph.add_node(id = person_id1, name = name1)
            combo_list = [str(d2['id']) for d2 in v]
            if person_id not in set(combo_list):
                combo_list.append(person_id)
            for t in combinations(combo_list, 2):
                if t[0] != t[1] and (t[0], t[1]) not in set(graph.edges):
                    graph.add_edge(source = t[0], target = t[1])

In [ ]:
for (person_id, name) in graph.nodes[1:]:
    person_credits = tmdb_api_utils.get_movie_credits_for_person(person_id, vote_avg_threshold = 8.0)
    movie_cast = {}
    for d in person_credits:
        movie_cast[d['id']] = tmdb_api_utils.get_movie_cast(d['id'], limit = 3, exclude_ids = None)
    for k, v in movie_cast.items():
        for d1 in v:
            person_id1 = str(d1['id'])
            name1 = d1['name']
            if (person_id1, name1) not in set(graph.nodes):
                graph.add_node(id = person_id1, name = name1)
            combo_list = [str(d2['id']) for d2 in v]
            if person_id not in set(combo_list):
                combo_list.append(person_id)
            for t in combinations(combo_list, 2):
                if t[0] != t[1] and (t[0], t[1]) not in set(graph.edges):
                    graph.add_edge(source = t[0], target = t[1])

In [ ]:
for (person_id, name) in graph.nodes[1:]:
    person_credits = tmdb_api_utils.get_movie_credits_for_person(person_id, vote_avg_threshold = 8.0)
    movie_cast = {}
    for d in person_credits:
        movie_cast[d['id']] = tmdb_api_utils.get_movie_cast(d['id'], limit = 3, exclude_ids = None)
    for k, v in movie_cast.items():
        for d1 in v:
            person_id1 = str(d1['id'])
            name1 = d1['name']
            if (person_id1, name1) not in set(graph.nodes):
                graph.add_node(id = person_id1, name = name1)
            combo_list = [str(d2['id']) for d2 in v]
            if person_id not in set(combo_list):
                combo_list.append(person_id)
            for t in combinations(combo_list, 2):
                if t[0] != t[1] and (t[0], t[1]) not in set(graph.edges):
                    graph.add_edge(source = t[0], target = t[1])

In [ ]:
graph.write_edges_file()
graph.write_nodes_file()

In [ ]:
tst = None

In [ ]:
if not tst:
    print(1)

In [ ]:
movie_id = 455
api_key = 'e6c6d80656546a1dd359ab0b71ca6432'

In [ ]:
import http.client
import json
connection = http.client.HTTPSConnection("api.themoviedb.org")
connection.request("GET", "/3/movie/{}/credits?api_key={}".format(movie_id, api_key))
response = connection.getresponse()



In [ ]:
headers = response.getheaders()
body = response.read().decode()

In [ ]:
exclude_ids = [62230, 231610]
limit = 5
movie_cast = json.loads(body)

In [ ]:
movie_cast = None

In [ ]:
if movie_cast is not None:
    output = [d for d in movie_cast['cast']]
    if exclude_ids is not None:
        output = [d for d in output if d['id'] not in exclude_ids]
    if limit is not None:
        output = [d for d in output if d['order'] < limit]
    #output = output[0:limit]
else:
    output = []
output

#else:
#    print([d for d in cast['cast']])
#print(type(cast))
#print(read)
#print(type(read))
#msg = response.msg
#print(msg)
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint("Headers: {}".format(headers))
#print("Status: {} and reason: {}".format(response.status, response.reason))



In [ ]:
output = [d for d in movie_cast['cast']]
if exclude_ids is not None:
    output = [d for d in output if d['id'] not in exclude_ids]
if limit is not None:
    output = [d for d in output if d['order'] < limit]
output

In [ ]:
person_id = 455
api_key = 'e6c6d80656546a1dd359ab0b71ca6432'

In [ ]:
connection = http.client.HTTPSConnection("api.themoviedb.org")
connection.request("GET", "/3/person/{}/movie_credits?api_key={}&language=en-US".format(person_id, api_key))
response = connection.getresponse()

In [ ]:
body = response.read().decode()
person_credits = json.loads(body)

In [ ]:
vote_avg_threshold = 7.7

In [ ]:
person_credits = None
person_credits

In [ ]:
person_credits

In [ ]:
person_credits['cast']

In [ ]:
if person_credits is not None:
    output = [{k : v for k, v in d.items() if k in {'credit_id', 'title', 'vote_average', 'id'}} for d in person_credits['cast']]
    if vote_avg_threshold is not None:
        output = [d for d in output if d['vote_average'] >= vote_avg_threshold]
else:
    output = []
output

In [ ]:
connection.close()

In [ ]:
E = [('e1', 'e2'), ('e2', 'e3'), ('e1', 'e3'), ('e3', 'e4'), ('e2', 'e5')]
print(E)

In [ ]:
d = {}

for (e0, e1) in E:
    if e0 not in d.keys():
        d[e0] = 1
    else:
        d[e0] += 1
    if e1 not in d.keys():
        d[e1] = 1
    else:
        d[e1] += 1

        
print(d)
print(max(d.values()))

{k : v for k, v in d.items() if v == max(d.values())}

In [ ]:
t = ('3', 'Marina')
A = [('1', 'Ramin'), ('2', 'Ramina')]
print(A)
print(tuple(reversed(t)))
print(set(A))

In [ ]:
A.append(t)
A

In [ ]:
if t not in set(A):
    A.extend(t)
A

In [ ]:
t = ('4', 'Michelle')
if t not in set(A):
    A.append(t)
A

Let $
X_{i \in I}^{j \in J} = \left[
    \begin{array}{ccc}
    x_1^1&\cdots&x_1^d\\
    \vdots&x_i^j&\vdots\\
    x_n^1&\cdots&x_n^d
    \end{array}
\right] , \left[\begin{array}{c}I = {1,..., n}\\J = {1,..., d}\end{array} \right].
$

Then, $E[X_{i \in I}^{j \in J}] = \left[\begin{array}{c}E[X^1]\\\vdots\\E[X^d]\end{array} \right].$





In [ ]:
import sys

if not 'numpy' in sys.modules:
    !pip install numpy

import numpy as np

In [ ]:
a = np.array(
      [
            [
                  [10, 15, 1]
                , [ 5, 10, 3]
                , [10,  5, 3]
            ]
          , [
                  [ 5, 25, 2]
                , [20, 20, 3]
                , [ 5, 15, 1]
            ]
      ]
)
np.array(sorted(a.reshape(-1, 3), key = lambda v: [v[0], v[1]])).reshape(a.shape)


In [ ]:
np.rec.array(a)

In [ ]:
field_names = ['y', 'x', 'classifier']

In [ ]:
b = np.array([[[10, 15,  1],
        [ 5, 10,  3],
        [10,  5,  3]],

       [[ 5, 25,  2],
        [20, 20,  3],
        [ 5, 15,  1]]])

b

In [ ]:
array_with_labels = np.array([[[(label, num) for i, num in enumerate(l2) for j, label in enumerate(field_names) if i == j] for l2 in l1] for l1 in b], dtype = [('yax'), (), ()]
array_with_labels

In [ ]:
np.sort(array_with_labels, order = ['y', 'x'])

In [ ]:
[l2[0], l2[1], l2[2]]

In [ ]:
np.sort(a, axis = 0)

In [ ]:
import numpy as np
X = np.array([
    [5, 5, 4],
    [8, 1, 8],
    [0, 1, 5],
    [0, 3, 1],
    [6, 2, 5]
])

In [ ]:
np.linalg.inv(np.matmul(X, X.T))

In [ ]:
def find_max_common_label_in_cluster(clustering, y, k):
    import numpy as np
    z = np.array([-1] * k)
    for i in set(clustering):
        y_indices = np.where(clustering == i)[0]
        y_take = np.take(y, y_indices)
        y_filtered = np.extract(y_take != -1, y_take)
        if y_filtered.size:
            counts = np.bincount(y_filtered)
            z[i] = np.argmax(counts)            
    return z

In [ ]:
y = np.array([0,  0,  1,  0,  1,  0,  1,  0,  0,  0,  1,  0,  0,  1,  0,  0])

In [ ]:
counts = np.bincount(y)
counts

In [ ]:
np.argmin(counts)

In [ ]:
np.where(y == 1)[0]

In [ ]:
from collections import Counter

In [ ]:
ex2_demo_bags = [{'cat', 'dog', 'fish'},
                 {'red', 'blue'},
                 {'dog'},
                 {'one', 'two', 'dog', 'fish'}]


In [ ]:

[Counter(list(st)) for st in ex2_demo_bags]

In [ ]:
ex1_demo_input = ["This is a phrase; this, too, is a phrase",
                  "But this is another sentence",
                  "Hark",
                  "Come what may    <-- save those spaces, but not these -->",
                  "What did you say",
                  "Split into 3 (even without a space)",
                  "Okie dokie"]

In [ ]:
def clean(s):
    from re import finditer
    pattern = r"[a-z]+('[a-z])?[a-z]*"
    return [match.group(0) for match in finditer(pattern, s.lower())]

In [ ]:
def gen_bag_of_words(sents):
    assert isinstance(sents, list)
    for sentence in sents:
        for word in clean(sentence) if word not in STOPWORDS]
    
# Demo:
gen_bag_of_words(ex1_demo_input)

In [ ]:
import re

In [ ]:
ex0_demo_input = ["  This is a phrase; this, too, is a phrase. But this is another sentence.",
                  "Hark!",
                  "    ",
                  "Come what may    <-- save those spaces, but not these -->    ",
                  "What did you say?Split into 3 (even without a space)? Okie dokie."]

In [ ]:
result = []
for element in ex0_demo_input:
    result.extend([s.strip() for s in re.split(r'\.|\?|\!', element)])
    result = [s for s in result if s]
result

In [ ]:
clustering = np.array([ 8, 5, 5, 8, 6, 7,  4, 1, 8, 7, 5,  8, 8])
y = np.array([-1, 5, 6, 9, 6, 7, -1, 1, -1, 7, 6, -1, 3])
k = 10
z = np.array([-1] * k)

In [ ]:
#from collections import Counter
for i in set(clustering):
    y_indices = np.where(clustering == i)[0]
    y_take = np.take(y, y_indices)
    y_filtered = np.extract(y_take != -1, y_take)
    if y_filtered.size:
        counts = np.bincount(y_filtered)
        z[i] = np.argmax(counts)
        #most_common = Counter(y_filtered).most_common()
        #if most_common[0][1] == most_common[-1][1]:
        #    z[i] = min(most_common)[0]
        #else:
        #    z[i] = most_common[0][0]
z

In [ ]:
Y = np.array([4, 2, 1, 1, 3, 2, 4, 2, 2, 2, 4])
y = np.array([2, 3, 5, 6])

In [ ]:
p = Y[y]
p

In [ ]:
counts = np.bincount(p)
counts

In [ ]:
np.argmax(counts)

In [ ]:
import numpy as np
X = np.array([
    [5, 5, 4],
    [8, 1, 8],
    [0, 1, 5],
    [0, 3, 1],
    [6, 2, 5]
])

w = np.array([
    [8, 5, 1],
    [7, 8, 6]
])

x = np.array(
    [8, 5, 1]
)

In [ ]:
X - w[:,np.newaxis]

In [ ]:
X[:,np.newaxis] - w

In [ ]:
np.sum(X - x, axis = 1)

In [ ]:
np.sqrt(np.sum((X - x)**2, axis=1))

In [ ]:
np.linalg.norm(X - x, axis = 1)

In [ ]:
np.argsort(np.sqrt(np.sum((X - x)**2, axis=1)))[0:2]

In [ ]:
l = [(1, 2), (4, 3), (7, 8), (10, 11)]

In [ ]:
import operator

In [ ]:
x = 0

In [ ]:
[operator.iadd(x + a + b, x + a + b) for a, b in l]

In [ ]:
f = operator.itemgetter(1, 3)

In [ ]:
f(l)

In [ ]:
res = []
aux = set()
for x in nums:
  if x not in aux:
    res.append(x)
    aux.add(x)

In [ ]:
import numpy as np
A = np.array([[0, 1, 0, 1, 0, 1],
              [1, 0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [1, 1, 1, 0, 1, 1],
              [0, 0, 0, 1, 0, 0],
              [1, 0, 0, 1, 0, 0]], dtype=int)

In [ ]:
(np.multiply(A.dot(A), A).sum(axis = 1)/6).astype(int)

In [ ]:
I = [0, 1, 2, 3, 2, 3]

In [ ]:
J = [1, 0, 3, 2, 3, 2]

In [ ]:
from scipy.sparse import coo_matrix

In [ ]:
n = max(I) + 1

In [ ]:
V = np.ones(len(I), dtype=int)
V

In [ ]:
A = coo_matrix((V, (I, J)), shape=(n, n))
print(A)

In [ ]:
A.sum_duplicates()
print(A)

In [ ]:
A = (A > 0).astype(int).todense()
print(A)

In [ ]:
C = A.dot(A) * A
print(C.sum(axis = 1)/6)

In [ ]:
import numpy as np

In [ ]:
A = np.zeros((2, 2))
A[1,1] = 1

In [ ]:
A

In [ ]:
from itertools import groupby

In [ ]:
l = [[2,5], [2, 5], [3,4]]

In [ ]:
[k for k, g in groupby(l)]

In [ ]:
s = [1, 2, 3, 4]
from itertools import permutations, combinations


In [ ]:
{t : 1 for t in list(permutations(s, 2))}

In [ ]:
{t : 1 for t in list(combinations(s, 2))}

In [ ]:
inputData = ''

In [ ]:
import math

In [ ]:
'\n'.join(
    [
        str(int(math.log(int(line), 2).is_integer())) 
        for line in inputData.splitlines()[1:]
    ]
)

In [ ]:
def codeHere():
    lines = [int(line) for line in inputData.splitlines()]
    bool_vals = [(x != 0) and (x & (x - 1) == 0) for x in lines[1:]]
    output = list(map(int, bool_vals))
    output = [str(x) for x in output]
    return '\n'.join(output)

In [ ]:
import numpy as np

In [ ]:
v = np.asarray([True, False, True, False])

In [ ]:
# if you don't specify the replacements
# then you get back the indices of `True` values
np.where(v)

In [ ]:
# `v` should be a `numpy` array
np.where(v, 4, 5)

In [ ]:
[xv if c else yv
 for c, xv, yv in zip(condition, x, y)]

In [ ]:
s = 'We analyzed found the issue in Garbage Collection which crashed the JVM.\r\n\r\n\xa0\r\n\r\n\xa0\r\n\r\n_Stack: [0x00007f0b58ff1000,0x00007f0b590f1000],\xa0 sp=0x00007f0b590ef120,\xa0 free space=1016k_\r\n\r\n_Native frames: (J=compiled Java code, j=interpreted, Vv=VM code, C=native code)_\r\n\r\n_V\xa0 [libjvm.so+0x8b9e4f]\xa0 MethodData::clean_extra_data(BoolObjectClosure)+0x1cf_\r\n\r\n_V\xa0 [libjvm.so+0x63c582]\xa0 '

In [ ]:
import re

In [ ]:
re.sub(r'(\\r)|(\\n)|(\\xa0)', '', s)

In [ ]:
dic = {
    'A' : ['b', 'a', 'a', 'd', 'd', 'c'],
    'B' : [3, 1, 1, 4, 1, 4],
    'C' : [3, 2, 2, 4, 2, 5],
    'D' : [4, 1, 1, 1, 1, 6]
}
df = pd.DataFrame(dic)
df

In [ ]:
df = (
    df
        .assign(A = lambda x: pd.Categorical(x['A'], categories = ['b', 'd', 'c', 'a'], ordered = True))
        .sort_values('A')
)

In [ ]:
dic = {
    'a': {
        'b': [1, 2], 
        'c': [3, 4]
    }, 
    'A': {
        'B': [10, 20], 
        'C': [30, 40]
    }
}
s = (
    pd.DataFrame(dic)
        .stack()
        .explode()
        .reset_index()
        .filter(['level_1', 'level_0', 0])
        .rename(columns = {'level_1' : 'col_1', 'level_0' : 'col_2', 0 : 'col_3'})
)
s

In [ ]:
import pandas as pd


def my_function(df):
    kwargs = {
        "quantity": df.apply(lambda x: -x["quantity"] if "SELL" in str(x["action"]) else x["quantity"], axis = 1),
        "price (usd)": df.apply(lambda x: 0 if "SELL" in str(x["action"]) else x["price (usd)"], axis = 1),
    }
    return df.assign(**kwargs)


input_df = pd.DataFrame({"product": ["APPLE", "APPLE", "BANANA"],
                         "action": ["  SELL  ", "  BUY  ", "  SELL  "],
                         "quantity": [1, 2, 3],
                         "price (usd)": [3, 5, 8],
                         })

result_df = my_function(input_df)
result_df

In [ ]:
expected_df = pd.DataFrame({"product": ["APPLE", "APPLE", "BANANA"],
                            "action": ["  SELL  ", "  BUY  ", "  SELL  "],
                            "quantity": [-1, 2, -3],
                            "price (usd)": [0, 5, 0],
                            })
expected_df

In [ ]:
    numbers = [25, 24, 26, 45, 25, 23, 50, 51]
    count = len(numbers)
    filtered = [num for num in numbers if 25 <= num <= 50]
    count = len(numbers) - len(filtered)
    filtered

If a polynomial function has integer coefficients, then every rational zero will have the form

$$
\frac{p}{q}
$$

where `p` is a factor of the constant and `q` is a factor of the leading coefficient.

In [ ]:
import numpy as np
A = np.array ([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]
              ], dtype=float)

print ("A[:, :] ==\n", A)
print ("\na0 := A[:, 0] ==\n", A[:, 0])
print ("\na1 := A[:, 2:3] == \n", A[:, 2])

print ("\nAdd columns 0 and 2?")
a0 = A[:, 0]
a1 = A[:, 2]
print (a0 + a1)

In [ ]:
import numpy as np

In [ ]:
np.convolve([2, 3, 5], [7, 6, 4])

In [ ]:
np.convolve([2, 3, 5], [7, 6, 4], 'same')

In [ ]:
X = np.array([[1,1,0], [1,0,0], [0,0,1]])

In [ ]:
np.ones(X.shape) * -1

In [ ]:
X

In [ ]:
kernel = np.ones(X.shape) * -1

In [ ]:
from scipy.signal import convolve2d
convolve2d(X, kernel, 'same')

In [ ]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

In [ ]:
def insertLevelOrder(arr, root, i, n): 
      
    # Base case for recursion  
    if i < n: 
        temp = TreeNode(arr[i])  
        root = temp  
  
        # insert left child  
        root.left = insertLevelOrder(arr, root.left, 
                                     2 * i + 1, n)  
  
        # insert right child  
        root.right = insertLevelOrder(arr, root.right, 
                                      2 * i + 2, n) 
    return root

In [ ]:
def inOrder(root): 
    if root != None: 
        inOrder(root.left)  
        print(root.val,end=" ")  
        inOrder(root.right) 
        
if __name__ == '__main__': 
    arr = [1, 2, 3, 4, 5, 6, 6, 6, 6] 
    n = len(arr) 
    root = None
    root = insertLevelOrder(arr, root, 0, n)  
    inOrder(root)

In [ ]:
stk = [root]

In [ ]:
node = stk.pop()

In [ ]:
inOrder(node)

In [ ]:
node = stk.pop()

In [ ]:
J, S = "aA", "aAAbbbb"

In [ ]:
print(root.left)

In [ ]:
class Solution:
    def numJewelsInStones(self, J: str, S: str) -> int:
        count = 0
        for j in J:
            for s in S:
                if s == j:
                    count += 1
        return count

In [ ]:
for c in S:
    print(c)

In [ ]:
set(S)

In [ ]:
set(J)

In [ ]:
sum([1 for c in S if c in J])

In [ ]:
Jli = list(set(list(J)))
count = 0
for j in Jli:
	count += S.count(j)
count

In [ ]:
sum([S.count(i) for i in J])

In [ ]:
num = 14

In [ ]:
sum([int(d) + 1 for d in bin(num)[2:]]) - 1

In [ ]:
bin(num)[2:]

In [ ]:
len(b := bin(num)[2:]) + b.count('1') - 1

In [ ]:
myList = [1, 1, 2, 2, 2, 3]
output = {}

for item in myList:
    if item not in output:
        output[item] = 0
    output[item] += 1

output

In [ ]:
from string import ascii_letters, digits
import timeit

In [ ]:
words = list(ascii_letters+digits)*10**5
len(words)

In [ ]:
start_time = timeit.default_timer()
from collections import defaultdict
d = defaultdict(int)
for w in words: d[w] += 1
elapsed = timeit.default_timer() - start_time
elapsed

In [ ]:

%timeit words
d = {i:words.count(i) for i in words}

In [ ]:
mycode = ''' 
d = defaultdict(int)
for w in words: d[w] += 1
'''

In [ ]:
mycode = ''' 
#words = list(digits)*10**5

answers = [0, 2, 2, 2]

d = Counter(answers)
'''

In [ ]:
mysetup = '''
from collections import Counter
#from string import ascii_letters, digits
'''

In [ ]:
timeit.timeit(setup = mysetup, stmt = mycode, number = 7)

In [ ]:
from collections import Counter
%timeit words
d = Counter(words)

In [ ]:
answers = [0, 2, 2, 2]
%timeit answers
from collections import Counter

d = Counter(answers)

In [ ]:
words=[0, 2, 2, 2]
{i:words.count(i) for i in words}

In [ ]:
start_time = timeit.default_timer()
# code you want to evaluate
elapsed = timeit.default_timer() - start_time

In [ ]:
from itertools import groupby as gb
from math import ceil
answers = [1,0,1,0,0]  
lst = [(len(list(v)), g) for g, v in gb(sorted(answers))]

[(len(list(v)), g) for g, v in gb(sorted(answers))]

In [ ]:
[ceil(t[0]/(t[1] + 1)) * (t[1] + 1) for t in lst]

In [ ]:
from collections import Counter
Counter(answers)

In [ ]:
sum([ceil(v/(k + 1)) * (k + 1) for k, v in Counter(answers).items()])

In [ ]:
[len(answers) + 1 for g, v in gb(sorted(answers))]

In [ ]:
# For every grouping of K rabbits, where K is the total number of rabbits that had the 
# same answer, we can deduce that the minimum number of rabbits is equal to Math.ceil(K/(answer + 1)) * (answer + 1).

In [ ]:
from collections import Counter
answers = [0,0,1,1,1]
l1 = [list(v) for g, v in gb(sorted(answers))]
c = Counter(answers)
c

In [ ]:
sum((c[i] + i) / (i + 1) * (i + 1) for i in c)

In [ ]:
[divmod(len(l), 2) for l in l1]

In [ ]:
answers = [1,0,1,0,0]
#sum(len(answers)) + divmod(len(answers), 2)[1]
#divmod(len(answers), 2)[1]

In [ ]:
from itertools import groupby as gb
l = list(gb(answers, key = lambda k: k))
#list(list(gb(answers))[0][1])

list(l[0][1])

In [ ]:
[(t[0], list(t[1])) for t in l]

In [ ]:
import pandas as pd
df=[['7','4','5','',''],["","","",'7','4'],['9','4','7','8','4'],["","","",'7','5'],['4','8','5','4','6'],['4','9','41','74','20'],['4','Super','yes','0','9']]
df=pd.DataFrame(df)
df.astype('str')
df.iloc[1 : 4,:].apply(lambda x : ''.join(x)).to_frame().transpose()#.reset_index()#.pivot(index = 'index', columns = 'index', values = '0')

In [ ]:


# this is an important step because we need a column to groupby
#df['gb'] = 'gb'

start_end = [[1,3],[4,5]]

#df.columns = ['0', '1', '2', '3', '4', 'gb']

new_df = pd.DataFrame()

for l in start_end:
    
    #temp_df = df.iloc[l[0] : l[1] + 1,:].groupby(['gb']).agg(lambda x : ''.join(x)).reset_index(drop = True)
    temp_df = df.iloc[l[0] : l[1] + 1,:].apply(lambda x : ''.join(x)).to_frame().transpose()
    
    new_df = pd.concat([new_df, temp_df], ignore_index = True)

new_df

In [ ]:
start_end = [[1,3],[4,5]]

new_df = pd.DataFrame()

for l in start_end:
    
    temp_df = df.iloc[l[0] : l[1] + 1,:].apply(lambda x : ''.join(x)).to_frame().transpose()
    
    new_df = pd.concat([new_df, temp_df], ignore_index = True)

new_df

In [ ]:
from itertools import product
{
  sum(x ** 2 for x in xs) : xs
  for xs in product(range(7), repeat=3)
  if sum(x ** 2 for x in xs) % 8 in {3, 4, 0}
}

In [ ]:
[
    (x, f(x) , g(x)) 
    for (x, f(x), g(x)) in (
        (x, f(x), g(fx))
        for (x, f(x)) in (
            (x, f(x)) 
            for x in bigList
        )
        if f(x) < p
    )
    if g(x) < q
]

In [ ]:
def xs(x, y):
    return x - y

iter1 = [1, 2, 3]
iter2 = [4, 6, 8, 9]

iter(list(map(xs, iter1, iter2)))

In [ ]:
import re
pattern = r'base=\"(?P<groupname>[0-9]{1,10})\"'
s = '<div aim="" base="1005260" class="item card cb23 eb02 rb3 d1" ele="2" imgur="kMBzbLb" quantity="" release="" res="1005260" type="1"><div class="content"></div><a class="ab" hash="" href="/cards/1005260-vegito-peerless-combination" title="Peerless Combination - Vegito"><table><tr><td class="dokkan"></td><td class="element"></td></tr><tr><td class="rarity"><i>ssr</i></td><td class="lock off"><i class="material-icons off"></i><i class="material-icons on"></i></td></tr></table></a><div class="dv">11921</div></div>'
int(re.search(pattern, s).group('groupname'))
